In [2]:
import numpy as np

# problem2:Data Partitioning

In [6]:
def Partitioning_MNIST(data, labels):
    """Return training_data, training_labels, validation_data, validation_labels of MNIST"""

    rng = np.random.default_rng()
    data_copy, labels_copy = rng.permutation(data, axis=0), rng.permutation(labels, axis=0)

    training_data = data_copy[:50000]
    training_labels = labels_copy[:50000]

    validation_data = data_copy[50000:]
    validation_labels = labels_copy[50000:]

    return training_data, training_labels, validation_data, validation_labels

In [ ]:
def Partition_SPAM(data, labels):
    """Return training_data, training_labels, validation_data, validation_labels of SPAM"""

    rng = np.random.default_rng()
    data_copy, labels_copy = rng.permutation(data, axis=0), rng.permutation(labels, axis=0)

    num = data["training_data"].shape * 0.2

    training_data = data_copy[:num]
    training_labels = labels_copy[:num]

    validation_data = data_copy[num:]
    validation_labels = labels_copy[num:]

    return training_data, training_labels, validation_data, validation_labels

In [ ]:
def Partitioning_CIFAR10(data, labels):
    """Return training_data, training_labels, validation_data, validation_labels of CIFAR10"""

    rng = np.random.default_rng()
    data_copy, labels_copy = rng.permutation(data, axis=0), rng.permutation(labels, axis=0)

    training_data = data_copy[:5000]
    training_labels = labels_copy[:5000]

    validation_data = data_copy[5000:]
    validation_labels = labels_copy[5000:]

    return training_data, training_labels, validation_data, validation_labels